In [1]:
import pandas as pd
from owlready2 import *
import types

path = '../data/output.xlsx'

df =  pd.read_excel(path, dtype={'CN_Code': str, 'NST 2007': str})
df

,# CRM,CN_Code,CN18_omschrijving,Antimony,Beryllium,Chromium,Cobalt,Cokingcoal,Fluorspar,PhosphateRock,...,Ytterbium,Yttrium,NST 2007,NST EN,NST DE,NST FR,CN 2018 EN,CN 2018 DE,CN 2018 FR,_merge
0,0,01012100,fokpaarden van zuiver ras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,01.8,Live animals,Lebende Tiere,Animaux vivants,Pure-bred breeding horses,"Zuchtpferde, reinrassig",Chevaux reproducteurs de race pure,both
1,0,01012910,slachtpaarden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,01.8,Live animals,Lebende Tiere,Animaux vivants,Horses for slaughter,Pferde zum Schlachten,Chevaux destinés à la boucherie,both
2,0,01012990,"paarden, levend (m.u.v. fokpaarden van zuiver ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,01.8,Live animals,Lebende Tiere,Animaux vivants,"Live horses (excl. for slaughter, pure-bred fo...","Pferde, lebend (ausg. zum Schlachten, reinrass...",Chevaux vivants (à l’excl. des animaux reprodu...,both
3,0,01013000,"ezels, levend",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,01.8,Live animals,Lebende Tiere,Animaux vivants,Live asses,"Esel, lebend","Ânes, vivants",both
4,0,01019000,"muildieren en muilezels, levend",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,01.8,Live animals,Lebende Tiere,Animaux vivants,Live mules and hinnies,"Maultiere und Maulesel, lebend","Mulets et bardots, vivants",both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9540,0,99309900,Boordprovisie (goederen van de hoofdstukken 25...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9541,0,99312400,Goederen die zijn geleverd voor de op de insta...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9542,0,99312700,Goederen die zijn geleverd voor de werking van...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9543,0,99319900,Goederen die zijn geleverd voor de op de insta...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


#### Create dummy materials ontology

In [2]:
# create and populate ontology
dummy = get_ontology('http://www.semanticweb.org/geofluxus/ontologies/2022/dummy_materials')
print(dummy.base_iri)

http://www.semanticweb.org/geofluxus/ontologies/2022/dummy_materials#


In [3]:
import re

# print(df.columns[3:-8])
mat_names = df.columns[3:-8]

with dummy:
  dummy.metadata.versionInfo.append('1.1.0')
  dummy.metadata.label.append('Material Ontology (dummy)')
  dummy.metadata.seeAlso.append('https://www.geofluxus.com/')

  class Material(Thing):
    pass

  class CNCode(DataProperty, FunctionalProperty):
    domain = [Material]
    range = [str]

  class NSTCode(DataProperty, FunctionalProperty):
    domain = [Material]
    range = [str]

  class CECode(DataProperty, FunctionalProperty):
    domain = [Material]
    range = [str]
    
  class RowIndex(DataProperty, FunctionalProperty):
    domain = [Material]
    range = [str]
    
  class Weights(DataProperty, FunctionalProperty):
    domain = [Material]
    range = [float]

  for mat in mat_names:
    label_txt = re.findall('[A-Z][^A-Z]*', mat)
    merged = ''
    for l in label_txt:
      merged += f' {l}'
    merged = merged.lstrip()
    
    mat = types.new_class(mat, (Material,))
    mat.label = [locstr(f'{merged}', lang='en')]


close_world(Material)
# save ontology to file
dummy.save(file = '../ontologies/dummy.owl', format = "rdfxml")